# 센텀 의료기관 네이버 검색광고 ROI 분석
## 1단계: 네이버 검색광고 API 데이터 수집 및 정제

**프로젝트 목표**: 네이버 검색광고의 효율성 검증 및 ROI 분석

**분석 일정**: 2025-10-27 ~ 2025-11-30

**태스크**: 네이버 검색광고 API를 통해 캠페인, 키워드, 광고그룹 별 성과 데이터 수집 및 초기 정제

---

### 📚 API 참고 자료
- **Naver Search Ads API Docs**: [https://naver.github.io/searchad-apidoc/#/guides](https://naver.github.io/searchad-apidoc/#/guides)
- **Python Nevada Library**: [https://github.com/taegyumin/python_nevada](https://github.com/taegyumin/python_nevada)
- **Base URL**: `https://api.searchad.naver.com` ✨
- **경로 형식**: `/searchad/{resource}.naver` ✨
- **인증**: X-API-KEY, X-Customer, X-TIMESTAMP, X-SIGNATURE (HMAC-SHA256)

## 0. 라이브러리 및 설정

In [8]:
import os
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import requests
import json
import hmac
import hashlib
import time
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns

project_root = Path('..').resolve()
scripts_dir = (project_root / 'scripts')
if scripts_dir.exists():
    sys.path.insert(0, str(scripts_dir))

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'AppleGothic'

print("✓ 라이브러리 로드 완료")

✓ 라이브러리 로드 완료


In [9]:
# API 인증 정보 로드
info_path = Path('../data/info.csv')
if not info_path.exists():
    raise FileNotFoundError(f'API 정보 파일을 찾을 수 없습니다: {info_path}')

api_info = pd.read_csv(info_path)
if api_info.empty:
    raise ValueError('data/info.csv 파일이 비어있습니다.')

row = api_info.loc[api_info['media'] == 'nsa']
if row.empty:
    raise ValueError('네이버 검색광고 API 정보가 data/info.csv에 없습니다.')

naver_api_key = str(row['key'].squeeze()).strip()
naver_customer_id = None
if 'customer_id' in row.columns:
    val = str(row['customer_id'].squeeze()).strip()
    if val:
        naver_customer_id = val
if naver_customer_id is None and 'scr' in row.columns:
    scr_val = str(row['scr'].squeeze()).strip()
    if scr_val.isdigit():
        naver_customer_id = scr_val
if naver_customer_id is None:
    raise ValueError('Customer ID 정보를 찾을 수 없습니다. data/info.csv에 customer_id 컬럼을 추가하세요.')

naver_secret_key = os.environ.get('NAVER_SECRET_KEY')
secret_source = '환경 변수' if naver_secret_key else None
if not naver_secret_key:
    candidate_secret = None
    if 'secret' in row.columns:
        candidate_secret = str(row['secret'].squeeze()).strip()
    elif 'scr' in row.columns:
        scr_val = str(row['scr'].squeeze()).strip()
        if scr_val and not scr_val.isdigit():
            candidate_secret = scr_val
    if candidate_secret:
        naver_secret_key = candidate_secret
        secret_source = 'info.csv'

print('✓ Naver Search Ads API 정보 로드 완료')
print(f'  - API Key (앞 6자): {naver_api_key[:6]}...')
print(f'  - Customer ID: {naver_customer_id}')
if secret_source:
    print(f'  - Secret Key: {secret_source}에서 로드됨')
else:
    print('  - Secret Key: ⚠️ 미설정 (환경 변수 NAVER_SECRET_KEY 또는 data/info.csv에 secret 컬럼 필요)')


✓ Naver Search Ads API 정보 로드 완료
  - API Key (앞 6자): 010000...
  - Customer ID: 2398132
  - Secret Key: info.csv에서 로드됨


## 1. API 키 로드

## 2. 네이버 검색광고 API 클래스 정의 (Nevada 방식)

### 인증 방식 (python_nevada 기반)
- `X-API-KEY`: 액세스라이선스
- `X-Customer`: 고객 ID (숫자)
- `X-TIMESTAMP`: Unix timestamp (밀리초)
- `X-SIGNATURE`: HMAC-SHA256(secret_key, "{METHOD} {PATH} {TIMESTAMP}")

### 경로 형식
- `/searchad/apiservice.naver` - 시간 조회 (테스트)
- `/searchad/campaigns.naver` - 캠페인 목록
- `/searchad/campaigns/{id}/adgroups.naver` - 광고 그룹
- `/searchad/campaigns/{id}/adgroups/{id}/keywords.naver` - 키워드


> 아래 코드는 python_nevada 방식을 참고하여 구현한 커넥터 클래스이며, 실제 API 호출은 다음 코드 셀에서 수행됩니다.


In [10]:
class NaverSearchAdsConnector:
    """
    네이버 검색광고 API 커넥터 (python_nevada 방식)
    
    - Base URL: https://api.searchad.naver.com
    - 경로 패턴: /searchad/{resource}.naver
    - 인증 헤더: X-API-KEY, X-Customer, X-Timestamp, X-Signature
    """

    def __init__(self, api_key: str, secret_key: str, customer_id: str):
        self.api_key = api_key
        self.secret_key = secret_key
        self.customer_id = customer_id
        self.base_url = 'https://api.searchad.naver.com'

    def _generate_signature(self, method: str, path: str, timestamp: str) -> str:
        message = f"{method} {path} {timestamp}"
        signature = hmac.new(
            self.secret_key.encode('utf-8'),
            message.encode('utf-8'),
            hashlib.sha256
        ).hexdigest()
        return signature

    def _build_headers(self, method: str, path: str) -> dict:
        timestamp = str(int(time.time() * 1000))
        signature = self._generate_signature(method, path, timestamp)
        return {
            'X-API-KEY': self.api_key,
            'X-Customer': self.customer_id,
            'X-Timestamp': timestamp,
            'X-Signature': signature,
            'Content-Type': 'application/json'
        }

    def _request(self, method: str, path: str, payload: dict | None = None) -> dict | None:
        url = f"{self.base_url}{path}"
        headers = self._build_headers(method, path)
        data = None if payload is None else json.dumps(payload, ensure_ascii=False)
        try:
            if method == 'GET':
                response = requests.get(url, headers=headers, timeout=10)
            elif method == 'POST':
                response = requests.post(url, headers=headers, data=data, timeout=10)
            else:
                raise ValueError(f'Unsupported HTTP method: {method}')

            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            resp = e.response
            if resp is not None:
                print(f"❌ HTTP {resp.status_code}: {resp.text}")
            else:
                print(f"❌ HTTP 오류: {e}")
            return None
        except requests.exceptions.RequestException as e:
            print(f"❌ 요청 실패: {e}")
            return None

    def get_datetime(self) -> dict | None:
        return self._request('GET', '/searchad/apiservice.naver')

    def get_campaigns(self) -> dict | None:
        return self._request('GET', '/searchad/campaigns.naver')

    def get_keywords(self, campaign_id: str, ad_group_id: str) -> dict | None:
        path = f'/searchad/campaigns/{campaign_id}/adgroups/{ad_group_id}/keywords.naver'
        return self._request('GET', path)

    def get_statistics(self, payload: dict) -> dict | None:
        return self._request('POST', '/searchad/stats.naver', payload)


if naver_secret_key:
    nsa_api = NaverSearchAdsConnector(
        api_key=naver_api_key,
        secret_key=naver_secret_key,
        customer_id=naver_customer_id
    )
    print('✓ Naver Search Ads API 커넥터 초기화 완료')
else:
    nsa_api = None
    print('⚠️ Secret Key가 설정되지 않아 API 호출을 건너뜁니다. 환경 변수 NAVER_SECRET_KEY를 설정하세요.')



✓ Naver Search Ads API 커넥터 초기화 완료


## 3. 캠페인 데이터 수집

In [12]:
# 캠페인 목록 조회
if nsa_api is None:
    print("⚠️ Secret Key가 설정되지 않아 API 호출을 건너뜁니다.")
    campaigns_response = None
else:
    print("📊 캠페인 목록 조회 중...")
    campaigns_response = nsa_api.get_campaigns()
    if campaigns_response:
        print("✓ 캠페인 조회 완료")
        print("응답 샘플 (처음 200자):")
        print(json.dumps(campaigns_response, indent=2, ensure_ascii=False)[:200])
    else:
        print("❌ 캠페인 조회 실패")


📊 캠페인 목록 조회 중...
❌ HTTP 404: 
❌ 캠페인 조회 실패


## 4. 통계 데이터 수집

최근 30일 기간의 일별 성과 지표 수집

In [15]:
# 조회 기간 설정 (최근 30일)
end_date = datetime.now().date()
start_date = end_date - timedelta(days=30)

print(f"📅 조회 기간: {start_date} ~ {end_date}")

# 통계 데이터 조회
if nsa_api is None:
    print("⚠️ Secret Key가 설정되지 않아 통계 조회를 건너뜁니다.")
    stats_response = None
else:
    print("📊 통계 데이터 조회 중...")
    stats_payload = {
        "timeRange": {
            "startDate": str(start_date),
            "endDate": str(end_date)
        },
        "timeUnit": "DAY",
        "fields": [
            "impressions",
            "clicks",
            "cost",
            "conversions",
            "sales",
            "ctr",
            "cpc",
            "roas"
        ]
    }
    stats_response = nsa_api.get_statistics(stats_payload)

    if stats_response:
        print("✓ 통계 데이터 조회 완료")
        print("응답 샘플 (처음 300자):")
        print(json.dumps(stats_response, indent=2, ensure_ascii=False)[:300])
    else:
        print("❌ 통계 데이터 조회 실패")


📅 조회 기간: 2025-10-03 ~ 2025-11-02
📊 통계 데이터 조회 중...
❌ HTTP 404: 
❌ 통계 데이터 조회 실패


## 5. 데이터 정제 및 기초 검증

In [ ]:
def validate_api_response(response_data):
    """
    API 응답 데이터 검증
    """
    if not response_data:
        return False, "응답 없음"
    
    if isinstance(response_data, dict):
        if 'error' in response_data or 'errors' in response_data:
            return False, f"API 에러: {response_data.get('error', response_data.get('errors'))}"
    
    return True, "정상"

# 응답 검증
if campaigns_response:
    is_valid, msg = validate_api_response(campaigns_response)
    print(f"캠페인 응답 검증: {msg}")

if stats_response:
    is_valid, msg = validate_api_response(stats_response)
    print(f"통계 응답 검증: {msg}")

## 6. 결과 저장

In [ ]:
import os
from pathlib import Path

# 저장 경로 생성
output_dir = Path('../data/raw')
output_dir.mkdir(parents=True, exist_ok=True)

# 시간 스탐프
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 응답 데이터 JSON으로 저장
if campaigns_response:
    campaign_file = output_dir / f'campaigns_{timestamp}.json'
    with open(campaign_file, 'w', encoding='utf-8') as f:
        json.dump(campaigns_response, f, ensure_ascii=False, indent=2)
    print(f"✓ 캠페인 데이터 저장: {campaign_file}")

if stats_response:
    stats_file = output_dir / f'stats_{timestamp}.json'
    with open(stats_file, 'w', encoding='utf-8') as f:
        json.dump(stats_response, f, ensure_ascii=False, indent=2)
    print(f"✓ 통계 데이터 저장: {stats_file}")

## 7. 검증 기준 체크리스트

### 태스크 검증 기준
- ✅ API 데이터 성공 수집
- ✅ 결측치율 < 5%
- ✅ 시계열 데이터 정합성 확인

In [ ]:
print("\n" + "="*60)
print("📋 태스크 검증 기준 체크리스트")
print("="*60)

# 1. API 데이터 수집 여부
api_success = campaigns_response is not None and stats_response is not None
status_1 = '✓ PASS' if api_success else '✗ FAIL'
print(f"\n1️⃣  API 데이터 성공 수집: {status_1}")

# 2. 결측치율
print(f"\n2️⃣  결측치율 < 5%: (통계 데이터 확보 후 검증)")

# 3. 시계열 정합성
print(f"\n3️⃣  시계열 데이터 정합성: (통계 데이터 확보 후 검증)")

print("\n" + "="*60)
print("📝 다음 단계")
print("="*60)
print("✓ Task 2: 스마트로그 전환 데이터 수집")
print("  - 사용자 여정 경로 데이터")
print("  - 온/오프라인 전환 이벤트")
print("  - 매체별 유입 추적")
print("="*60)


📋 태스크 검증 기준 체크리스트

1️⃣  API 데이터 성공 수집: ✗ FAIL

2️⃣  결측치율 < 5%: (통계 데이터 확보 후 검증)

3️⃣  시계열 데이터 정합성: (통계 데이터 확보 후 검증)

📝 다음 단계
✓ Task 2: 스마트로그 전환 데이터 수집
  - 사용자 여정 경로 데이터
  - 온/오프라인 전환 이벤트
  - 매체별 유입 추적
